# **Ch08. 텍스트분석 실습 #7**

**Ch08-09. 한글 텍스트 처리 - 네이버 영화 평점 감성 분석**

* 데이터 다운로드 :  https://github.com/e9t/nsmc 에서 `ratings.txt`, `ratings_train.txt`, `ratings_test.txt` 내려받기


In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = False)

Mounted at /content/gdrive


## import data

In [4]:
import pandas as pd

train_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/2020-TAVE-ML-Practice-Study/data/ratings_train.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [5]:
# 0과 1 라벨 값 비율 확인
# 1: 긍정, 0: 부정

train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

1. 'document'칼럼에 일부 존재하는 Null값을 공백으로 대체
2. 숫자의 경우 단어적 의미로 부족-> 're'모듈을 사용해 공백으로 변환

In [7]:
import re

# Null을 공백으로
train_df = train_df.fillna(' ')

# 숫자를 공백으로 (정규 표현식으로 \d 는 숫자를 의미함.) 
train_df['document'] = train_df['document'].apply(lambda x:re.sub(r"\d+"," ", x))

# 테스트 데이터 셋 로딩 후 동일한 과정 수행
test_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/2020-TAVE-ML-Practice-Study/data/ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )

# id 칼럼 삭제
train_df.drop('id', axis=1, inplace=True) 
test_df.drop('id', axis=1, inplace=True)


## **TF-IDF Vectorization**
1. Twitter 클래스의 morphs( ) 메소드 이용하여 문장을 형태소 단어 형태로 토큰화하는 tw_tokenizer( )함수 생성
2. 사이킷런의 TfidfVectorizer이용하여 TF-IDF 모델 생성(tokenizer 는 위의 함수 이용)

In [9]:
pip install konlpy

     |████████████████████████████████| 19.4MB 37.1MB/s 
     |████████████████████████████████| 92kB 5.4MB/s 
     |████████████████████████████████| 3.8MB 43.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [10]:
from konlpy.tag import Twitter

twitter = Twitter()
def tw_tokenizer(text):
  # 입력 인자로 들어온 text를 형태소 단어로 토큰화하여 list 객체 반환
  tokens_ko = twitter.morphs(text)
  return tokens_ko

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [12]:
# 로지스틱 회귀로 분류 기반의 감성 분석 수행
lg_clf = LogisticRegression(random_state=0)

# 하이퍼라미터 C의 최적화를 위해 GridSearchCV이용
params = {'C':[1, 3.5, 4.5, 5.5, 10]}
grid_cv = GridSearchCV(lg_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params_, round(grid_cv.best_score_, 4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
 

{'C': 4.5} 0.8327


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [13]:
from sklearn.metrics import accuracy_score

# train데이터 적용했던 TfidfVectorizer이용하여 test데이터도 피처변환
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ', accuracy_score(test_df['label'], preds))

Logistic Regression 정확도:  0.94852
